# Cleaning Data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./data/movies_mojo_uptoZ.csv", 
                   dtype = {'widest_release_theaters' : str,
                            'producer' : str,
                            'actors' : str,
                            'director' : str,
                            'distributor' : str
                           })
data = data.drop_duplicates()
data.rename(columns={'runtime_(mins)': 'runtime',
                     'widest_release_theaters' : 'theaters',
                     'domestic_total_gross' : 'total_gross',
                     'production_budget' : 'budget'
                    }, inplace=True)
meta = pd.read_csv("./data/metacritic.csv")
meta = meta.drop_duplicates()

In [3]:
meta['release_date'] = pd.to_datetime(meta.release_date, format="%b %d, %Y", errors = 'coerce')
data['release_date'] = pd.to_datetime(data.release_date, format="%Y-%m-%d %H:%M:%S", errors = 'coerce')

In [4]:
# Removing lines with NaNs for 'theaters' and 'release_date' and 'budget' and 'runtime' and 'total_gross'
data = data[ pd.notnull(data['theaters']) ]
data = data[ pd.notnull(data['release_date']) ]
data = data[ pd.notnull(data['budget']) ]
data = data[ pd.notnull(data['runtime']) ]
data = data[ pd.notnull(data['total_gross']) ]

In [5]:
# Converting 'theaters' str to float
data['theaters'] = data['theaters'].map(lambda x: float(x.replace(',','')))

In [6]:
# Converting 'budget' str to float
def production_to_int(s):
    money_dict = { 'million' : 1000000 }
    s = s.replace(',','').replace('$','').strip().split()
    if len(s) == 1:
        return float(s[0])
    elif len(s) == 2:
        return float(s[0])*money_dict[s[1]]
    else:
        return None

data['budget'] = data['budget'].map(lambda x: production_to_int(x))

In [7]:
data['month'] = data['release_date'].map(lambda x: x.month)
data['year'] = data['release_date'].map(lambda x: x.year)

In [8]:
# Adjusting for inflation: 1999 - 2016
inflation_dict = {
                  1999 : 1.43,
                  2000 : 1.38,
                  2001 : 1.34,
                  2002 : 1.32,
                  2003 : 1.29,
                  2004 : 1.26,
                  2005 : 1.22,
                  2006 : 1.18,
                  2007 : 1.15,
                  2008 : 1.11,
                  2009 : 1.11,
                  2010 : 1.09,
                  2011 : 1.06,
                  2012 : 1.04,
                  2013 : 1.02,
                  2014 : 1.01,
                  2015 : 1.00,
                  2016 : 1.00
                 }
for years in range(1900,1999):
    inflation_dict[years] = 1.00
# Years before 1999 are unadjusted: Not using those gross values
data['inflation'] = data['year'].map(lambda x: inflation_dict[x])
data['total_gross'] = data['total_gross']*data['inflation']
data['budget'] = data['budget']*data['inflation']

In [9]:
data['gross_per_theater'] = data['total_gross']/data['theaters']

In [10]:
def get_row_names(names):
    if not names:
        return None
    if pd.isnull(names):
        return None
    clean_list = []
    names_list = names.split(',')
    for name in names_list:
        name = name.strip()
        if name != "(executive)" and name != "":
            clean_list.append(name)
    return clean_list

In [11]:
def get_df(df, column_name):
    headers = ['movie_title', column_name, 'release_date', 'total_gross']
    new_df = pd.DataFrame()
    for i in range(len(df)):
        movie = df.movie_title.iloc[i]
        total_gross = df.total_gross.iloc[i]
        release_date = df.release_date.iloc[i]
        item_list = get_row_names(df[column_name].iloc[i])
        if item_list:
            for item in item_list:
                row = pd.DataFrame([[movie, item, release_date, total_gross]], columns=headers)
                new_df = new_df.append(row, ignore_index=True)
    return new_df

In [12]:
# Calculate average total_gross per actor per movie for movies released by that date
# Then for each movie, take the maximum average total_gross == actor_score
# Finally, merge with the existing data frame by movie_title
def get_score(df, column_name, column_score):
    df2=pd.merge(df[['movie_title',column_name,'total_gross','release_date']],
                 df[['movie_title',column_name,'total_gross','release_date']],
                 on=column_name)
    df2=df2[df2['release_date_x']>df2['release_date_y']]
    df2 = df2.groupby(['movie_title_x',column_name,'total_gross_x','release_date_x']).mean()
    df2 = df2.reset_index()
    df2 = df2.rename(columns={'movie_title_x': 'movie_title', 'total_gross_y': column_score})
    df2 = df2.groupby(['movie_title']).max().reset_index()
    df2 = df2[['movie_title',column_score]]
    return df2

In [13]:
data_actors = get_df(data, "actors")
data_actors = data_actors[ pd.notnull(data_actors['total_gross']) ]
actors_score = get_score(data_actors, 'actors','actors_score')

In [14]:
data_director = get_df(data, "director")
data_director = data_director[ pd.notnull(data_director['total_gross']) ]
director_score = get_score(data_director, 'director','director_score')

In [15]:
data_producer = get_df(data, "producer")
data_producer = data_producer[ pd.notnull(data_producer['total_gross']) ]
producer_score = get_score(data_producer, 'producer','producer_score')

In [16]:
data_writers = get_df(data, "writers")
data_writers = data_writers[ pd.notnull(data_writers['total_gross']) ]
writers_score = get_score(data_writers, 'writers','writers_score')

In [17]:
data_composers = get_df(data, "composers")
data_composers = data_composers[ pd.notnull(data_composers['total_gross']) ]
composers_score = get_score(data_composers, 'composers','composers_score')

In [18]:
data_cinematographers = get_df(data, "cinematographers")
data_cinematographers = data_cinematographers[ pd.notnull(data_cinematographers['total_gross']) ]
cinematographers_score = get_score(data_cinematographers, 'cinematographers','cinematographers_score')

In [25]:
# Create data frames with scores for actors, actors+producers, etc.
# Measure the length of the (inner)joined data frame
data_actors_score = pd.merge(data, actors_score, on='movie_title')
print len(data_actors_score)
data_producer_score = pd.merge(data_actors_score, producer_score, on='movie_title')
print len(data_producer_score)
data_director_score = pd.merge(data_producer_score, director_score, on='movie_title')
print len(data_director_score)
data_writers_score = pd.merge(data_director_score, writers_score, on='movie_title')
print len(data_writers_score)
data_composers_score = pd.merge(data_writers_score, composers_score, on='movie_title')
print len(data_composers_score)
data_cinematographers_score = pd.merge(data_composers_score, cinematographers_score, on='movie_title')
print len(data_composers_score)

2352
1650
935
551
421
421


In [20]:
def get_merged_data(df):
    md = pd.merge(meta, df, on = 'movie_title')
    return md

In [21]:
#md_actors = get_merged_data(data_actors_score)
#md_actors.to_csv("./data/md_actors.csv", index = False)

In [22]:
md_producer = get_merged_data(data_producer_score)
md_producer.to_csv("./data/md_actor_producer.csv", index = False)

In [23]:
#md_all = get_merged_data(data_cinematographers_score)
#md_all.to_csv("./data/md_all.csv", index = False)